# Análisis de la Tableta de Cerro Macareno

Este cuaderno documenta el análisis exploratorio y avanzado de la Tableta de Cerro Macareno. Se transcriben los datos de la tablilla, se analizan distintas clasificaciones (inicialmente con múltiples estados) y se concluye que, en términos prácticos, el sistema es binario (vertical vs. horizontal) conservando además la categoría de casillas en blanco. Se evalúa la influencia de la orientación de lectura (horizontal vs. vertical) mediante análisis de correlación espacial, FFT, reducción de dimensionalidad (PCA y t-SNE) y clustering difuso (Fuzzy C-means).

**Nota:** Para ejecutar este cuaderno, asegúrate de tener en la carpeta `data/` las imágenes `tablilla.jpg` y `tablilla_saez_2006.jpg`, así como el archivo Excel `tableta.xlsx`.

## 1. Imagen inicial de la tablilla

![Tableta de Cerro Macareno](data/tablilla.jpg)

## 1.2. Imagen de referencia adicional

![Otra vista de la tableta](data/tablilla_saez_2006.jpg)

Imagen extraída de:  
**Sáez Uribarri, Í. (2006):** “La tableta de Cerro Macareno: Análisis exploratorio de datos en torno a una pieza de arqueología”, *Spal* 15: 259-266.  
[DOI: https://dx.doi.org/10.12795/spal.2006.i15.13](https://dx.doi.org/10.12795/spal.2006.i15.13)

## 2. Creación del DataFrame transcrito

Cargamos el archivo Excel `tableta.xlsx` con la transcripción de la tablilla (8 filas x 14 columnas).

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style='whitegrid', font_scale=1.1)

# Cargar el archivo Excel
df = pd.read_excel("data/tableta.xlsx", header=0)
print("DataFrame original (orientación horizontal):")
display(df)

## 3. Transformación a modelo binario

Inicialmente se consideraron varias categorías (vertical, horizontal, inclinadas, etc.). Sin embargo, los análisis preliminares mostraron que el sistema se comporta de forma binaria, pero es fundamental conservar las casillas en blanco (valor 5). Por ello, se transforma la matriz de la siguiente manera:
- Si el valor es exactamente 5, se mantiene como 5 (blanco).
- Si el valor es menor a 2.5 y distinto de 5, se asigna 1 (vertical).
- Si el valor es mayor o igual a 2.5 y distinto de 5, se asigna 3 (horizontal).

In [ ]:
def transformar_a_binario(grid, umbral=2.5):
    """
    Convierte la matriz original a un modelo binario conservando las casillas en blanco (valor 5):
      - Si el valor == 5, se mantiene 5 (blanco).
      - Si el valor < umbral y distinto de 5, se asigna 1 (vertical).
      - Si el valor >= umbral y distinto de 5, se asigna 3 (horizontal).
    """
    grid_binario = np.where(grid == 5, 5, np.where(grid < umbral, 1, 3))
    return grid_binario

# Convertir a matriz NumPy
grid_original = df.values.astype(float)
grid_binario = transformar_a_binario(grid_original, umbral=2.5)

print("Matriz binaria (1 = vertical, 3 = horizontal, 5 = vacío):")
print(grid_binario)

## 4. Análisis en orientación Horizontal

Realizamos varios análisis: suma por filas y columnas, análisis de autocorrelación espacial (promedio de vecinos y FFT), reducción de dimensionalidad (PCA y t-SNE) y clustering difuso (Fuzzy C-means).

In [ ]:
def calcular_promedio_vecinos(grid):
    """Calcula el promedio de los 8 vecinos (vecindad de Moore) para cada celda."""
    n_rows, n_cols = grid.shape
    neighbor_avg = np.empty((n_rows, n_cols))
    for i in range(n_rows):
        for j in range(n_cols):
            vecinos = []
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    if di == 0 and dj == 0:
                        continue
                    ni, nj = i + di, j + dj
                    if 0 <= ni < n_rows and 0 <= nj < n_cols:
                        vecinos.append(grid[ni, nj])
            neighbor_avg[i, j] = np.mean(vecinos) if vecinos else np.nan
    return neighbor_avg

# Sumas por filas y columnas
sum_filas = np.sum(grid_binario, axis=1)
sum_columnas = np.sum(grid_binario, axis=0)

print("Suma de filas:", sum_filas)
print("Suma de columnas:", sum_columnas)

In [ ]:
from scipy.fft import fft, fftfreq

def analizar_periodicidad(vector, label=""):
    n = len(vector)
    yf = fft(vector)
    xf = fftfreq(n, d=1)
    magnitudes = np.abs(yf)
    
    plt.figure(figsize=(8,4))
    plt.stem(xf[:n//2], magnitudes[:n//2], basefmt=" ")
    plt.xlabel("Frecuencia")
    plt.ylabel("Magnitud")
    plt.title(f"Espectro FFT de {label} (n={n})")
    plt.show()
    return xf, magnitudes

print("\n--- Análisis FFT para filas (Horizontal) ---")
analizar_periodicidad(sum_filas, label="filas (Horizontal)")

print("\n--- Análisis FFT para columnas (Horizontal) ---")
analizar_periodicidad(sum_columnas, label="columnas (Horizontal)")

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def aplicar_PCA(grid):
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(grid)
    print("Componentes principales (PCA):")
    print(pca_result)
    plt.figure(figsize=(6,5))
    plt.scatter(pca_result[:,0], pca_result[:,1], c='blue')
    plt.title("PCA de la tabla (Horizontal)")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()
    return pca_result

def aplicar_tSNE(grid, orientacion="Horizontal", perplexity=3):
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    tsne_result = tsne.fit_transform(grid)
    plt.figure(figsize=(6,5))
    plt.scatter(tsne_result[:,0], tsne_result[:,1], c='green')
    plt.title(f"t-SNE de la tabla ({orientacion})")
    plt.xlabel("Dim 1")
    plt.ylabel("Dim 2")
    plt.show()
    return tsne_result

print("\n--- PCA (Horizontal) ---")
aplicar_PCA(grid_binario)

print("\n--- t-SNE (Horizontal) ---")
aplicar_tSNE(grid_binario, orientacion="Horizontal", perplexity=3)

In [ ]:
from sklearn.preprocessing import StandardScaler
import skfuzzy as fuzz

def fuzzy_cmeans_clustering(grid, k=2, m=2, error=0.005, maxiter=1000):
    n_rows, n_cols = grid.shape
    datos = []
    for i in range(n_rows):
        for j in range(n_cols):
            datos.append([i, j, grid[i, j]])
    datos = np.array(datos)
    
    scaler = StandardScaler()
    datos_scaled = scaler.fit_transform(datos)
    
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
        datos_scaled.T, c=k, m=m, error=error, maxiter=maxiter, init=None)
    
    cluster_labels = np.argmax(u, axis=0)
    cluster_grid = cluster_labels.reshape(n_rows, n_cols)
    
    print(f"Para k = {k}: Fuzzy Partition Coefficient (FPC) = {fpc:.4f}")
    return cntr, u, cluster_grid, fpc, datos

def plot_fuzzy_clusters(cluster_grid, k):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cluster_grid, annot=True, fmt="d", cmap="viridis", cbar=True)
    plt.title(f"Clusters (Fuzzy C-means) para k = {k} (Horizontal)")
    plt.xlabel("Columna")
    plt.ylabel("Fila")
    plt.show()

print("\n--- Clustering difuso (Horizontal) ---")
cntr, u, cluster_grid, fpc, datos = fuzzy_cmeans_clustering(grid_binario, k=2)
plot_fuzzy_clusters(cluster_grid, k=2)

## 5. Análisis en orientación Vertical

Para evaluar si la orientación de lectura influye en la interpretación, se transpone la matriz binaria y se repite el análisis.

In [ ]:
# Transponer la matriz para orientación Vertical
grid_vertical = grid_binario.T

# Sumas por filas y columnas en orientación Vertical
sum_filas_vert = np.sum(grid_vertical, axis=1)
sum_columnas_vert = np.sum(grid_vertical, axis=0)

print("[Orientación Vertical]")
print("Suma de filas (Vertical):", sum_filas_vert)
print("Suma de columnas (Vertical):", sum_columnas_vert)

# Análisis FFT en orientación Vertical
print("\n--- Análisis FFT para filas (Vertical) ---")
analizar_periodicidad(sum_filas_vert, label="filas (Vertical)")

print("\n--- Análisis FFT para columnas (Vertical) ---")
analizar_periodicidad(sum_columnas_vert, label="columnas (Vertical)")

# PCA y t-SNE en orientación Vertical
print("\n--- PCA (Vertical) ---")
aplicar_PCA(grid_vertical)

print("\n--- t-SNE (Vertical) ---")
aplicar_tSNE(grid_vertical, orientacion="Vertical", perplexity=3)

# Clustering difuso en orientación Vertical
print("\n--- Clustering difuso (Vertical) ---")
cntr_v, u_v, cluster_grid_v, fpc_v, datos_v = fuzzy_cmeans_clustering(grid_vertical, k=2)
plot_fuzzy_clusters(cluster_grid_v, k=2)

## 6. Conclusiones

- **Estructura binaria:** La transformación a un modelo que conserva tres estados (1 = vertical, 3 = horizontal, 5 = vacío) simplifica la interpretación, revelando bloques diferenciados en la tablilla.
- **Autocorrelación espacial:** Las sumas por filas y columnas, junto con el análisis FFT, indican que la distribución de las marcas no es aleatoria y presenta patrones espaciales.
- **Reducción de dimensionalidad:** Tanto PCA como t-SNE muestran agrupaciones latentes que confirman la robustez de la estructura, independientemente de la orientación.
- **Clustering difuso:** El clustering con k=2 produce una partición robusta (FPC ≈ 0.70), apoyando la hipótesis de que la Tableta se divide en dos secciones (por ejemplo, presencia vs. ausencia de un fenómeno).

**Interpretación arqueoastronómica (Hipotética):** Aunque no se detectan periodicidades astronómicas evidentes, la existencia de bloques diferenciados abre la posibilidad de que la Tableta registre información espacial o temporal, lo que podría relacionarse con patrones astronómicos. Se recomienda comparar estos resultados con datos astronómicos para avanzar en esta hipótesis.